In [1]:
#!pip install datasets>=1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 16.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.5 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: levenshtein
    Found existing installation: Levenshtein 0.20.9
    Uninstalling Levenshtein-0.20.9:
      Successfully uninstalled Levenshtein-0.20.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python-levenshtein 0.20.9 requires L

In [2]:
from datasets import load_dataset, load_metric, Audio
import os
import torch

In [3]:
#timit = load_dataset("timit_asr")
#minds = load_dataset("PolyAI/minds14",name="en-US", split="train[:100]")
#minds = load_dataset("librispeech_asr",'clean',split="train[:1000]")#, name="en-US", split="train")
#minds = load_dataset("common_voice", "tr", split="train+validation")
#common_voice
#librispeech_asr 
#print(timit)
minds = load_dataset("superb","asr", split="train[:10%]")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/28539 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2703 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2620 [00:00<?, ? examples/s]

Dataset superb downloaded and prepared to /root/.cache/huggingface/datasets/superb/asr/1.9.0/b8183f71eabe8c559d7f3f528ab37a6a21ad1ee088fd3423574cecad8b3ec67e. Subsequent calls will reuse this data.


In [4]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

In [5]:
minds

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 2854
})

In [6]:
#minds["train"]

In [7]:
#minds = minds.remove_columns(['english_transcription', 'intent_class', 'lang_id'])
minds = minds.remove_columns(['speaker_id', 'chapter_id', 'id'])

In [8]:
minds

Dataset({
    features: ['file', 'audio', 'text'],
    num_rows: 2854
})

In [9]:
#left_symb_list = ["$"]
#for index in range(10):
#    left_symb_list+=[str(index)]
#left_symb_list

In [10]:
#for index in range(563):
#    for symb in left_symb_list:
#        if symb in minds[index]["transcription"]:
#            print(minds[index]["transcription"])

In [12]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

minds = minds.map(remove_special_characters)

  0%|          | 0/2854 [00:00<?, ?ex/s]

In [13]:
minds

Dataset({
    features: ['file', 'audio', 'text'],
    num_rows: 2854
})

In [14]:
minds = minds.train_test_split(test_size=0.2)

In [15]:
minds["train"][0]
#minds[0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/39bad2659671b430f631158d09f2c89e593891e6d95eed4dbdda430301560fc0/LibriSpeech/train-clean-100/1594/135914/1594-135914-0096.flac',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/39bad2659671b430f631158d09f2c89e593891e6d95eed4dbdda430301560fc0/LibriSpeech/train-clean-100/1594/135914/1594-135914-0096.flac',
  'array': array([-0.00134277, -0.0017395 , -0.00067139, ...,  0.00106812,
          0.00106812,  0.00100708], dtype=float32),
  'sampling_rate': 16000},
 'text': 'who told her that ganem had rested well that night and that his disorder proceeding altogether from melancholy the cause being removed he would soon recover his health'}

In [16]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}
vocabs = minds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=minds.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'m': 0,
 'c': 1,
 'p': 2,
 'y': 3,
 'h': 4,
 'r': 5,
 'a': 6,
 "'": 7,
 't': 8,
 'u': 9,
 'l': 10,
 ' ': 11,
 'b': 12,
 'x': 13,
 'g': 14,
 'j': 15,
 's': 16,
 'q': 17,
 'o': 18,
 'v': 19,
 'i': 20,
 'e': 21,
 'd': 22,
 'z': 23,
 'n': 24,
 'w': 25,
 'k': 26,
 'f': 27}

In [18]:
len(vocab_dict)

28

In [19]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

30


In [20]:
import json
with open('./vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [21]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [22]:
tokenizer

PreTrainedTokenizer(name_or_path='', vocab_size=30, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'})

In [23]:
os.mkdir("./tokenizer_dir")

In [24]:
tokenizer.save_pretrained("./tokenizer_dir")

('./tokenizer_dir/tokenizer_config.json',
 './tokenizer_dir/special_tokens_map.json',
 './tokenizer_dir/vocab.json',
 './tokenizer_dir/added_tokens.json')

In [25]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [26]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [27]:
#minds["train"][0]["path"]

In [28]:
#minds["train"][0]["audio"]

In [29]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch


In [30]:
minds = minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=4)



#1:   0%|          | 0/571 [00:00<?, ?ex/s]

#0:   0%|          | 0/571 [00:00<?, ?ex/s]

#2:   0%|          | 0/571 [00:00<?, ?ex/s]

#3:   0%|          | 0/570 [00:00<?, ?ex/s]

#0:   0%|          | 0/143 [00:00<?, ?ex/s]

#1:   0%|          | 0/143 [00:00<?, ?ex/s]

#2:   0%|          | 0/143 [00:00<?, ?ex/s]

#3:   0%|          | 0/142 [00:00<?, ?ex/s]

In [31]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [32]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [33]:
wer_metric = load_metric("wer")


In [34]:
import numpy as np

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    #print({"wer": wer})
    return {"wer": wer}


In [35]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.codevectors', 'project_q.weight', 'project_hid.bias', 'project_hid.weight', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [36]:
model.freeze_feature_extractor()

In [37]:
os.mkdir("./model")

In [38]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./model",
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=50,
  logging_steps=50,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [40]:


from transformers import Trainer, TrainerCallback

#class PrinterCallback(TrainerCallback):
#    def on_log(self, args, state, control, logs=None, **kwargs):
#        _ = logs.pop("total_flos", None)
#        if state.is_local_process_zero:
#            print(logs)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=minds["train"],
    eval_dataset=minds["test"],
    tokenizer=processor.feature_extractor,
    #callbacks=[PrinterCallback]
)

Using amp fp16 backend


In [41]:
trainer.train()

***** Running training *****
  Num examples = 2283
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4290
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:207.)
  tensor = as_tensor(value)
/opt/conda/lib/python3.7/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning

Step,Training Loss,Validation Loss,Wer
50,8.970800,12.933688,0.996682
100,4.812500,3.458613,1.000000
150,3.054000,3.190386,1.000000
200,2.952400,3.040305,1.000000
250,2.920800,2.989865,1.000000
300,2.896000,2.950098,1.000000
350,2.880500,2.937628,1.000000
400,2.880200,2.936162,1.000000
450,2.866500,2.946703,1.000000
500,2.718500,2.392516,1.000000


***** Running Evaluation *****
  Num examples = 571
  Batch size = 16
***** Running Evaluation *****
  Num examples = 571
  Batch size = 16
***** Running Evaluation *****
  Num examples = 571
  Batch size = 16
***** Running Evaluation *****
  Num examples = 571
  Batch size = 16
***** Running Evaluation *****
  Num examples = 571
  Batch size = 16
***** Running Evaluation *****
  Num examples = 571
  Batch size = 16
***** Running Evaluation *****
  Num examples = 571
  Batch size = 16
***** Running Evaluation *****
  Num examples = 571
  Batch size = 16
***** Running Evaluation *****
  Num examples = 571
  Batch size = 16
***** Running Evaluation *****
  Num examples = 571
  Batch size = 16
Saving model checkpoint to ./model/checkpoint-500
Configuration saved in ./model/checkpoint-500/config.json
Model weights saved in ./model/checkpoint-500/pytorch_model.bin
Configuration saved in ./model/checkpoint-500/preprocessor_config.json
/opt/conda/lib/python3.7/site-packages/transformers/model

TrainOutput(global_step=4290, training_loss=0.5276449651012332, metrics={'train_runtime': 23746.0914, 'train_samples_per_second': 2.884, 'train_steps_per_second': 0.181, 'total_flos': 7.822566458000963e+18, 'train_loss': 0.5276449651012332, 'epoch': 30.0})

In [ ]:
import shutil
shutil.make_archive(output_filename, 'zip', dir_name)


In [42]:
trainer.save_model("./")

Saving model checkpoint to ./
Configuration saved in ./config.json
Model weights saved in ./pytorch_model.bin
Configuration saved in ./preprocessor_config.json


In [45]:
!zip model.zip ./model

updating: model/ (stored 0%)


In [47]:
!zip py_model.zip /kaggle/working/pytorch_model.bin

UsageError: Line magic function `%zip` not found.


In [55]:
%cd /kaggle/working

/kaggle/working


In [51]:
from IPython.display import FileLink
FileLink(r'py_model.zip')

/kaggle/working/py_model.zip

In [56]:
%ls

__notebook_source__.ipynb  preprocessor_config.json  training_args.bin
config.json                py_model.zip              vocab.json
model/                     pytorch_model.bin         wandb/
model.zip                  tokenizer_dir/


In [57]:
!tar -czvf my_work.zip -C . .

./
./vocab.json
./pytorch_model.bin
./.virtual_documents/
./my_work.zip
tar: ./my_work.zip: file changed as we read it
./config.json
./model/
./model/runs/
./model/runs/Jan11_10-38-06_04969468206b/
./model/runs/Jan11_10-38-06_04969468206b/events.out.tfevents.1673433645.04969468206b.23.0
./model/runs/Jan11_10-38-06_04969468206b/1673433645.9175594/
./model/runs/Jan11_10-38-06_04969468206b/1673433645.9175594/events.out.tfevents.1673433645.04969468206b.23.1
./model/checkpoint-4000/
./model/checkpoint-4000/trainer_state.json
./model/checkpoint-4000/pytorch_model.bin
./model/checkpoint-4000/scaler.pt
./model/checkpoint-4000/config.json
./model/checkpoint-4000/scheduler.pt
./model/checkpoint-4000/optimizer.pt
./model/checkpoint-4000/rng_state.pth
./model/checkpoint-4000/training_args.bin
./model/checkpoint-4000/preprocessor_config.json
./model/checkpoint-3500/
./model/checkpoint-3500/trainer_state.json
./model/checkpoint-3500/pytorch_model.bin
./model/checkpoint-3500/scaler.pt
./model/checkpo

In [58]:
FileLink(r'my_work.zip')

/kaggle/working/my_work.zip

In [60]:
import shutil
shutil.make_archive("tok", 'zip', "./tokenizer_dir")

'/kaggle/working/tok.zip'

In [62]:
!cd /kaggle/working

In [63]:
!zip -r file.zip /kaggle/working/tokenizer_dir

  adding: kaggle/working/tokenizer_dir/ (stored 0%)
  adding: kaggle/working/tokenizer_dir/vocab.json (deflated 49%)
  adding: kaggle/working/tokenizer_dir/tokenizer_config.json (deflated 34%)
  adding: kaggle/working/tokenizer_dir/special_tokens_map.json (deflated 39%)


In [65]:
FileLink(r'file.zip')

/kaggle/working/file.zip

In [ ]:
from IPython.display import FileLink
